In [2]:
from collections import deque
import pandas as pd
import numpy as np

In [3]:
from haversine import haversine

In [4]:
import math

In [5]:
node = pd.read_csv('/Users/juhong/Library/CloudStorage/OneDrive-아주대학교/대학교/3학년 2학기/캡스톤/DUI-traffic-stop-recommendation/Optimal_Choice_Algorithm/gangnam_node_2019.csv',encoding = 'cp949')

In [6]:
node.head()

,NODE_ID,NODE_TYPE,NODE_NAME,y좌표,x좌표
0,1220001100,108,동부간선도로-남부순환,127.090548,37.496133
1,1220000700,108,수서IC2,127.103633,37.491131
2,1220001500,108,동부간선도로-남부순환,127.077407,37.500554
3,1220025600,108,수서IC6,127.095864,37.492109
4,1220058500,108,올림픽대로-일반국도47,127.051645,37.528632


In [7]:
link = pd.read_csv('/Users/juhong/Library/CloudStorage/OneDrive-아주대학교/대학교/3학년 2학기/캡스톤/DUI-traffic-stop-recommendation/Optimal_Choice_Algorithm/gangnam_link_2019.csv',encoding = 'cp949')

In [8]:
link.head()

,Unnamed: 0,LINK_ID,F_NODE,T_NODE,ROAD_NAME_,LENGTH,TRAFFIC
0,0,1220110900,1220063900,1220060300,선릉로189길,126.845777,0
1,1,1220111000,1220060300,1220063900,선릉로189길,126.845777,0
2,2,1220109700,1220035000,1220035500,선릉로89길,168.723197,0
3,3,1220110500,1220045700,1220046500,봉은사로113길,68.769505,0
4,4,1220110300,1220044800,1220045700,봉은사로113길,78.915399,0


In [9]:
#dictionary를 활용하여 graph를 만들기 위한 sorting
node_sorted = node.sort_values(by=['NODE_ID'])

In [10]:
#node_reset_index
node_sorted = node_sorted.reset_index()
node_sorted

,index,NODE_ID,NODE_TYPE,NODE_NAME,y좌표,x좌표
0,535,1210012800,101,서초구-강남구경계(2),127.090283,37.463452
1,509,1210015200,105,서초구-강남구경계(3),127.065345,37.469391
2,528,1210016400,101,국악고교앞,127.048734,37.473734
3,517,1210017100,101,두상빌딩,127.046537,37.475846
4,529,1210019000,101,포이사거리,127.044592,37.478301
...,...,...,...,...,...,...
536,505,1220148500,101,율현2빗물펌프장,127.119206,37.471227
537,506,1220150600,104,구룡지하차도(동측),127.055029,37.474024
538,540,1230112800,105,서울시-성남시경계,127.127243,37.468450
539,538,2040015700,105,-,127.107924,37.462214


In [11]:
# index = node_sorted['NODE_ID'].searchsorted(1220000700)
# index
node.shape[0]

541

In [12]:
#dictionary를 활용한 undirected graph, 0과 651확인 필
graph = {}

check = [[0 for i in range(node.shape[0]+1)] for j in range(node.shape[0]+1)]

for i,data in link.iterrows():
    start,end = data['F_NODE'],data['T_NODE']
    index_start = node_sorted['NODE_ID'].searchsorted(start)
    index_end = node_sorted['NODE_ID'].searchsorted(end)
    
    #link의 시작노드, 끝노드 명을 index로 변환
    link['F_NODE'][i] = index_start
    link['T_NODE'][i] = index_end
    
    if(not check[index_start][index_end]):
        graph[index_start] = graph.get(index_start,[]) + [index_end]
        graph[index_end] = graph.get(index_end,[]) + [index_start]
    
    check[index_start][index_end] = 1
    check[index_end][index_start] = 1

graph

/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_5334/1881738529.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['F_NODE'][i] = index_start
/var/folders/d2/20dks6yj46b_077x0h5fmr740000gn/T/ipykernel_5334/1881738529.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  link['T_NODE'][i] = index_end


{526: [494],
 494: [526, 493],
 281: [286, 276, 288],
 286: [281, 290, 277],
 372: [378, 363, 359],
 378: [372, 389, 365, 391],
 363: [372, 352, 365, 353],
 352: [363, 340, 330],
 26: [27],
 27: [26, 416, 414, 25],
 416: [27, 44, 28],
 527: [425, 403, 439],
 425: [527, 429, 434, 444, 403],
 247: [37, 64, 245, 524],
 37: [247, 39, 524],
 414: [27, 419],
 531: [147, 176],
 147: [531, 141, 176],
 532: [165, 133, 154],
 165: [532, 174, 133, 154],
 529: [122, 33, 140],
 122: [529, 530, 108, 142],
 530: [122, 33, 118],
 108: [122, 128, 32],
 403: [527, 359, 398, 425, 66],
 44: [416, 428, 435],
 92: [94, 1],
 94: [92, 97, 100],
 36: [524, 224, 38],
 524: [36, 219, 523, 37, 247],
 196: [197, 202, 56],
 197: [196, 34, 188],
 224: [36, 209, 216],
 464: [451, 472, 476],
 451: [464, 444, 445],
 463: [472, 456, 444],
 472: [463, 464],
 488: [481, 505, 507],
 481: [488, 489, 468, 478, 480],
 489: [481, 504, 499],
 504: [489, 45, 509],
 505: [488, 496, 45],
 514: [512, 511],
 512: [514, 45, 510],
 51

In [13]:
class Cluster:
    def __init__(self,coordinate_center,radius):
        self.center = coordinate_center
        self.radius = radius
    
    def in_distance(self,coordinate):
        if(haversine(self.center,coordinate)*1000<=self.radius):
            return True
        else:
            return False

In [14]:
df_cluster = pd.read_csv('/Users/juhong/Library/CloudStorage/OneDrive-아주대학교/대학교/3학년 2학기/캡스톤/DUI-traffic-stop-recommendation/Optimal_Choice_Algorithm/cluster_data.csv',encoding = 'cp949')

set_cluster = [0 for i in range(df_cluster.shape[0])]

for i,data in df_cluster.iterrows():
    set_cluster[i] = Cluster((data['x좌표'],data['y좌표']),data['radius'])
    

In [17]:
def BFS(graph,start,dest,dist,visited):
    
    queue = deque([start])
    
    while queue:
        now = queue.popleft()
        
        if(dist[now]==5):
            return
        
        for i in graph[now]:
            if(not visited[i]):
                visited[i] = True
                queue.append(i)
                dist[i] = dist[now]+1
                if(dist[i] <= 4):
                    dest.append((start,i))
        
    return

In [18]:
short_check = [[0 for i in range(node.shape[0]+1)] for j in range(node.shape[0]+1)]

In [19]:
for i,data in link.iterrows():
    visited = [0 for i in range(node.shape[0])]
    dest = deque()
    dist = [0 for i in range(node.shape[0])]
    
    start_s = data['F_NODE']
    start_e = data['T_NODE']
    
    BFS(graph,start_s,dest,dist,visited)
    
    visited = [0 for i in range(node.shape[0])]
    dist = [0 for i in range(node.shape[0])]

    BFS(graph,start_e,dest,dist,visited)
    
    dest = list(set(dest))
    
    for v in dest:
        short_check[v[0]][v[1]] = True
        short_check[v[1]][v[0]] = True

In [20]:
#combination을 구하는 함수, 가장 처음의 combination부터 가장 마지막 combination까지 구하기 위함.
def nextComb(n,S):
    find = -1
    for i in range(0,len(S)):
        if S[i] != n-len(S)+i+1:
            find = i
    if find ==-1:
        return find

    S[find] = S[find]+1

    for j in range(find+1,len(S)):
        S[j] = S[find]+j-find

    return S

In [29]:
def check(a):
    length = len(a)
    for i in range(length):
        for j in range(i+1,length):
            first = link.loc[a[i]-1]
            second = link.loc[a[j]-1]
            
            #link 거리 5 안에 있음.
            check_1 = short_check[first['F_NODE']][second['F_NODE']]
            check_2 = short_check[first['F_NODE']][second['T_NODE']]
            check_3 = short_check[first['T_NODE']][second['F_NODE']]
            check_4 = short_check[first['T_NODE']][second['T_NODE']]
            
            if((check_1 and check_2) or (check_3 and check_4)):
                return 0         
    return 1

In [33]:
next = [1,2,3]
maximum = [-math.inf,0]

while(next!=-1):
    if(not check(next)):
        #next = nextComb(link.shape[0],next)
        continue
    
    sum = 0
    
    for i in next:
        #교통량
        traffic = link['TRAFFIC'][i-1]
        
        #링크의 시작노드, 끝노드
        start_node = link['F_NODE'][i-1]
        end_node = link['T_NODE'][i-1]
        
        #node의 데이터 저장
        start_node = node_sorted.loc[start_node]
        end_node = node_sorted.loc[end_node]
        
        #링크의 좌표는 각 node의 좌표의 평균
        coordinate = ((start_node['x좌표']+end_node['x좌표'])/2,(start_node['y좌표']+end_node['y좌표'])/2)
        
        #겹치는 클러스터의 수
        near = 1
        for i in range(df_cluster.shape[0]):
            if(set_cluster[i].in_distance(coordinate)):
                near += 1
        sum += traffic*near
        
    maximum = ([sum,next] if sum>maximum[0] else maximum)
    
    next = nextComb(link.shape[0],next)


KeyboardInterrupt: 

In [32]:
maximum

[1245, [48, 49, 50]]